In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import torch
import datasets
from datasets import Dataset
import accelerate
import pandas 

c:\Users\maheit\Desktop\book\Book_recommendation_system\book\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'datasets'

In [ ]:
# Load model
num_labels = len(mlb.classes_)  # Number of unique tropes
model = AutoModelForSequenceClassification.from_pretrained("KamilAin/bart-base-booksum", num_labels=num_labels)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",  # Directory for model checkpoints
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save model at the end of each epoch
    learning_rate=2e-5,  # Standard LR for transformers
    per_device_train_batch_size=8,  # Reduce if you face memory issues
    per_device_eval_batch_size=8,
    num_train_epochs=5,  # Adjust based on dataset size
    weight_decay=0.01,  # Regularization to prevent overfitting
    load_best_model_at_end=True,  # Load the best model (lowest eval loss)
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

# Train the model
trainer.train()


{'input_text': 'Summarize: Notes from Underground (pre-reform Russian: Записки изъ подполья; post-reform Russian: Записки из подполья, tr. Zapíski iz podpólʹya), also translated as Notes from the Underground or Letters from the Underworld, is an 1864 novella by Fyodor Dostoevsky. Notes is considered by many to be one of the first existentialist novels. It presents itself as an excerpt from the rambling memoirs of a bitter, isolated, unnamed narrator (generally referred to by critics as the Underground Man), who is a retired civil servant living in St. Petersburg. The first part of the story is told in monologue form, or the underground man\'s diary, and attacks emerging Western philosophy, especially Nikolay Chernyshevsky\'s What Is to Be Done? The second part of the book is called "Apropos of the Wet Snow" and describes certain events that appear to be destroying and sometimes renewing the underground man, who acts as a first person, unreliable narrator and anti-hero.', 'target_text':

In [ ]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer

# Load pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained("KamilAin/bart-base-booksum")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./bart_booksum_tropes",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs"
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,  # Ideally, use a separate validation dataset
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./bart_booksum_tropes")
tokenizer.save_pretrained("./bart_booksum_tropes")


c:\Users\maheit\dev\book\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\maheit\AppData\Local\Temp\ipykernel_3420\147608070.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.027700,0.091542
2,0.101200,0.078617
3,0.091800,0.071428
4,0.079200,0.068577


c:\Users\maheit\dev\book\lib\site-packages\transformers\modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./bart_booksum_tropes\\tokenizer_config.json',
 './bart_booksum_tropes\\special_tokens_map.json',
 './bart_booksum_tropes\\vocab.json',
 './bart_booksum_tropes\\merges.txt',
 './bart_booksum_tropes\\added_tokens.json',
 './bart_booksum_tropes\\tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load fine-tuned model
model_path = "./bart_booksum_tropes"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Input new book description
book_description = "James Anderson had a plan. Or half of one. All that matters is that he managed to do what his older brother, the famous Aaron Warner Anderson, never did: infiltrate Ark Island, the last refuge of The Reestablishment. In the past decade no outsider has breached the stronghold of the authoritarian regime, but James is in. In a prison cell, sure, but as far as James is concerned, a win is a win. It’s been ten years since the fall of The Reestablishment. Ten years since the notorious duo — Juliette Ferrars and Aaron Warner Anderson — led a worldwide rebellion and established the New Republic of the West. But after a decade of unsettling quiet, The Reestablishment is ready to make a devastating move, and they have the perfect person for the job."

# Tokenize input
inputs = tokenizer(f"Summarize: {book_description}", return_tensors="pt", max_length=512, truncation=True)

# Generate output
outputs = model.generate(**inputs, max_length=128)
tropes = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Predicted Tropes:", tropes)


c:\Users\maheit\dev\book\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Predicted Tropes: Fantasy, Young Adult, Adventure, Science fiction, War, Dystopian


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load fine-tuned model and tokenizer
model_path = "./bart_booksum_tropes"  # Ensure this is the correct path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Load ROUGE and BLEU metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

# Sample test dataset (replace with your actual test cases)
test_cases = [
    {
        "description": "Paedyn and Kai are reunited but face a terrible decision in this thrilling conclusion to the New York Times bestselling romantic fantasy trilogy perfect for fans of Sarah J. Maas and The Red Queen. Paedyn Gray and Kai Azer return to the Kingdom of Ilya… And Paedyn has a life-altering choice to make. Whatever she decides will determine her fate—and the fate of those around her—forever. In the ultimate battle of love and loyalty, who wins?",
        "expected_tropes": "Fantasy, Dystopia, Romance"
    },
    {
        "description": "From the #1 New York Times bestselling author of Just for the Summer comes a new playful yet deeply emotional contemporary romance.There might be no such a thing as a perfect guy, but Xavier Rush comes disastrously close. A gorgeous veterinarian giving Greek god vibes—all while cuddling a tiny kitten? Immediately yes. That is until Xavier opens his mouth and proves that even sculpted gods can say the absolute wrong thing. Like, really wrong. Of course, there’s nothing Samantha loves more than proving an asshole wrong…. . . unless, of course, he can admit he made a mistake. But after one incredible and seemingly endless date—possibly the best in living history—Samantha is forced to admit the truth, that her family is in crisis and any kind of relationship would be impossible. Samantha begs Xavier to forget her. To remember their night together as a perfect moment, as crushing as that may be.Only no amount of distance or time is nearly enough to forget that something between them. And the only thing better than one single perfect memory is to make a life—and even a love—worth remembering.",
        "expected_tropes": "Romance, Contemporary Romance"
    }
]

# Store predictions & references
predictions = []
references = []
all_pred_labels = []
all_true_labels = []



# confusion metric 

all_pred_tropes_set = set()
all_true_tropes_set = set()

for case in test_cases:
    # Tokenize input description
    inputs = tokenizer(f"Summarize: {case['description']}", return_tensors="pt", max_length=512, truncation=True)

    # Generate prediction
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=50, num_beams=5, early_stopping=True)

    # Decode output
    predicted_tropes = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Store results
    predictions.append(predicted_tropes)
    references.append([case["expected_tropes"]])  # BLEU expects list of references

    # Convert expected and predicted tropes into sets
    pred_tropes_set = set(predicted_tropes.lower().split(", "))
    true_tropes_set = set(case["expected_tropes"].lower().split(", "))

    # Merge all tropes for consistent binary encoding
    all_pred_tropes_set.update(pred_tropes_set)
    all_true_tropes_set.update(true_tropes_set)

# Compute ROUGE
rouge_scores = rouge.compute(predictions=predictions, references=references)

# Compute BLEU
bleu_score = bleu.compute(predictions=predictions, references=references)

# Convert full trope set into binary labels for P/R/F1
unique_tropes = list(all_pred_tropes_set | all_true_tropes_set)  # Union of all tropes

for case in test_cases:
    pred_tropes_set = set(case["expected_tropes"].lower().split(", "))
    true_tropes_set = set(case["expected_tropes"].lower().split(", "))

    pred_binary = [1 if trope in pred_tropes_set else 0 for trope in unique_tropes]
    true_binary = [1 if trope in true_tropes_set else 0 for trope in unique_tropes]

    all_pred_labels.append(pred_binary)
    all_true_labels.append(true_binary)

# Compute Precision, Recall, and F1-score
precision = precision_score(all_true_labels, all_pred_labels, average="macro")
recall = recall_score(all_true_labels, all_pred_labels, average="macro")
f1 = f1_score(all_true_labels, all_pred_labels, average="macro")

# Compute Jaccard Similarity (Better for Multi-Label Classification)
jaccard = jaccard_score(all_true_labels, all_pred_labels, average="macro")

# Print results
print("ROUGE Scores:", rouge_scores)
print("BLEU Score:", bleu_score)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Jaccard Score:", jaccard)


ROUGE Scores: {'rouge1': np.float64(0.125), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.125), 'rougeLsum': np.float64(0.125)}
BLEU Score: {'bleu': 0.0, 'precisions': [0.375, 0.14285714285714285, 0.0, 0.0], 'brevity_penalty': 0.8824969025845955, 'length_ratio': 0.8888888888888888, 'translation_length': 8, 'reference_length': 9}
Precision: 0.5
Recall: 0.5
F1 Score: 0.5
Jaccard Score: 0.5


c:\Users\maheit\dev\book\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\maheit\dev\book\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\maheit\dev\book\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\maheit\dev\book\lib\site-package

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Count label occurrences
label_counts = np.sum(train_dataset, axis=0)

# Plot label distribution
plt.figure(figsize=(12, 5))
plt.bar(range(len(mlb.classes_)), label_counts)
plt.xticks(range(len(mlb.classes_)), mlb.classes_, rotation=90)
plt.xlabel("Tropes")
plt.ylabel("Count")
plt.title("Label Distribution in Training Data")
plt.show()


NameError: name 'train_dataset' is not defined